In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexPrograms

In [ ]:
# initialize empty graph
graph = GraphOfConvexPrograms()

# vertex 1
v1 = graph.add_vertex("v1")
x1 = v1.add_variable(2)
c1 = np.array([-3, 0]) # center of the set
v1.add_constraint(cp.norm_inf(x1 - c1) <= 1)

# vertex 2
v2 = graph.add_vertex("v2")
x2 = v2.add_variable(2)
c2 = np.array([0, 2.5]) # center of the set
D2 = np.diag([.25, 2]) # scaling matrix
v2.add_constraint(cp.norm_inf(D2 @ (x2 - c2)) <= 1)

# vertex 3
v3 = graph.add_vertex("v3")
x3 = v3.add_variable(2)
c3 = np.array([3, 0]) # center of the set
v3.add_constraint(cp.norm2(x3 - c3) <= 1)
v3.add_constraint(x3 >= c3) # keep only top right part of the set

# vertex 4
v4 = graph.add_vertex("v4")
x4 = v4.add_variable(2)
c4 = np.array([0, -2.5]) # center of the set
D4 = np.diag([1, 2]) # scaling matrix
v4.add_constraint(cp.norm2(D4 @ (x4 - c4)) <= 1)

# vertex 5
v5 = graph.add_vertex("v5")
x5 = v5.add_variable(2)
c5 = np.array([.3, .3]) # center of the set
v5.add_constraint(cp.norm1(x5 - c5) <= 1)

In [ ]:
# add an edges vetween every pair of distinct vertices
for tail in graph.vertices:
    for head in graph.vertices:
        if tail != head:
            edge = graph.add_edge(tail, head)

            # edge cost is Euclidean distance
            edge.add_cost(cp.norm2(tail.variables[0] - head.variables[0]))

In [ ]:
# solve traveling salesman problem using Dantzig–Fulkerson–Johnson formulation
# all the subtour elimination constraints are included
prob = graph.solve_traveling_salesman(subtour_elimination=True)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
# instead of including all the subtour elimination constraints in the initial
# formulation, we can add them as needed

# function that computes a cycle with nonzero flow starting from a given vertex
def expand_cycle(graph, vertex, ye):
    cycle = [vertex]
    while True:
        out = graph.outgoing_edge_indices(cycle[-1])
        next_edge_idx = out[np.argmax(ye[out])]
        next_edge = graph.edges[next_edge_idx]
        next_vertex = next_edge.head
        if next_vertex == cycle[0]:
            break
        cycle.append(next_vertex)
    return cycle

# function that computes a subtour of minimum length
def shortest_subtour(graph, ye):
    vertices_left = graph.vertices
    subtour = graph.vertices
    while len(vertices_left) > 0:
        new_subtour = expand_cycle(graph, vertices_left[0], ye)
        if len(new_subtour) < len(subtour):
            subtour = new_subtour
        vertices_left = [vertex for vertex in vertices_left if vertex not in new_subtour]
    return subtour

# function that computes a linear constraint that eliminates a subtour
# inputs are yv and ye as required for a callback function
def subtour_elimination(yv, ye):
    ye_value = np.array([y.value for y in ye])
    subtour = shortest_subtour(graph, ye_value)
    if len(subtour) == graph.num_vertices():
        return []
    else:
        print(f"Eliminated subtour with {len(subtour)} vertices.")
        out = graph.outgoing_edge_indices(subtour)
        return [sum(ye[out]) >= 1]

In [ ]:
# solve traveling salesman using callback
prob = graph.solve_traveling_salesman(subtour_elimination=False, callback=subtour_elimination)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)